# A3b: Filtering  
Finn Hittson - fxh157  
CSDS 464  
Due: March 6th, 2023  

## 1 Filtering

### 1a. A moving average filter

Part (A):  
$y_M[n]$ has the form
$$
y_M[n]=\frac{1}{M}\sum_{k=0}^{M-1}x[n-k]=\frac{1}{M}(x[n]+x[n-1]+\cdots+x[n-M+1])
$$
$y_{M-1}[n-1]$ has the form
$$
y_{M-1}[n-1]=\frac{1}{M-1}\sum_{k=0}^{M-2}x[n-k-1]=\frac{1}{M}(x[n-1]+x[n-2]+\cdots+x[n-M+1])
$$
Distribute the $\frac{1}{M}$ in $y_M[n]$.
$$
y_M[n]=\frac{x[n]}{M}+\frac{x[n-1]}{M}+\cdots+\frac{x[n-M+1]}{M}
$$
Set $1-\lambda=\frac{1}{M}$. This makes $\lambda=\frac{M-1}{M}$. Reformat the expression.
$$
y_M[n]=(1-\lambda)x[n]+\frac{1}{M}\sum_{k=0}^{M-1}x[n-k-1]=(1-\lambda)x[n]+y_M[n-1]
$$
Note that $\lambda y_{M-1}[n-1]=y_M[n-1]$ so making this substitution gives us the desired result.
$$
y_M[n]=\lambda y_{M-1}[n-1]+(1-\lambda)x[n]
$$

Part (B):  
Write the derived expression in the form of summations.
$$
\frac{1}{M}\sum_{k=0}^{M-1}x[n-k]=\lambda\frac{1}{M-1}\sum_{k=0}^{M-2}x[n-k-1]+(1-\lambda)x[n]
$$
Approximate $\frac{1}{M-1}\sum_{k=0}^{M-2}x[n-k-1]\approx\frac{1}{M}\sum_{k=0}^{M-1}x[n-k-1]$ and simplify the expression.
$$
\frac{1}{M}\sum_{k=0}^{M-1}x[n-k]\approx\lambda\frac{1}{M}\sum_{k=0}^{M-1}x[n-k-1]+(1-\lambda)x[n]
$$
Let $y[n]=\frac{1}{M}\sum_{k=0}^{M-1}x[n-k]$ to get our desired result.
$$
y[n]=\lambda y[n-1]+(1-\lambda)x[n]
$$

Part (C):  
When $\lambda$ is increased, this causes $y[n-1]$ to become closer in value to $y[n]$ and has the opposite effect when $\lambda$ is decreased. 